## 1 - Setup

Define caminhos padronizados para artefatos desta versão (v2).

- MODELO_OUT → versions/v2-char-lm/models/modelo_char_lm_v2.keras
- MAPEAMENTOS_OUT → versions/v2-char-lm/mappings/mapeamentos_v2.pkl

Esses caminhos organizam os arquivos de treino e facilitam reuso.

In [ ]:
# SETUP_ARTIFACT_PATHS
from pathlib import Path
BASE_DIR = Path.cwd().resolve()
if BASE_DIR.name == 'notebooks':
    BASE_DIR = BASE_DIR.parent
MODELS_DIR = BASE_DIR / 'models'
MAPPINGS_DIR = BASE_DIR / 'mappings'
MODELS_DIR.mkdir(parents=True, exist_ok=True)
MAPPINGS_DIR.mkdir(parents=True, exist_ok=True)
MODELO_OUT = MODELS_DIR / 'modelo_char_lm_v2.keras'
MAPEAMENTOS_OUT = MAPPINGS_DIR / 'mapeamentos_v2.pkl'
print('Modelo:', MODELO_OUT)
print('Mapeamentos:', MAPEAMENTOS_OUT)


## 2 - Dados e Pre-Processamento

Carrega o corpus (ex.: Dom Casmurro) e aplica normalizações: lower() e compressão de espaços. Isso reduz ruído e ajuda modelos por caracteres a convergirem melhor.

In [ ]:
import tensorflow as tf, os, pickle
URL_LIVRO = 'https://www.gutenberg.org/files/55752/55752-0.txt'
caminho_arquivo = tf.keras.utils.get_file('dom_casmurro.txt', URL_LIVRO)
with open(caminho_arquivo, 'rb') as f:
    texto = f.read().decode('utf-8', errors='ignore')
texto = texto.lower()
texto = ' '.join(texto.split())
print('Tamanho do corpus (chars):', len(texto))


## 3 - Vocabulario e Janelas de Treino

Cria vocabulário e mapeamentos char→id/id→char (salvos em MAPEAMENTOS_OUT). Define SEQ_LEN=160 e um gerador one-hot de janelas e rótulos sob demanda.

In [ ]:
import numpy as np
SEQ_LEN = 160
caracteres = sorted(list(set(texto)))
char_to_id = {c: i for i, c in enumerate(caracteres)}
id_to_char = {i: c for i, c in enumerate(caracteres)}
vocab = len(caracteres)
with open(MAPEAMENTOS_OUT, 'wb') as f:
    pickle.dump({'char_to_id': char_to_id, 'id_to_char': id_to_char, 'tamanho_sequencia': SEQ_LEN}, f)
print('Vocabulario:', vocab, 'caracteres')

def batch_generator(texto, char2idx, seq_len=160, batch_size=256, step=1):
    n = len(texto) - seq_len
    V = len(char2idx)
    i = 0
    while True:
        X = np.zeros((batch_size, seq_len, V), dtype=np.float32)
        y = np.zeros((batch_size,), dtype=np.int32)
        for b in range(batch_size):
            idx = (i + b*step) % n
            seq = texto[idx: idx+seq_len
            nxt = texto[idx+seq_len]
            for t, ch in enumerate(seq):
                X[b, t, char2idx.get(ch, 0)] = 1.0
            y[b] = char2idx.get(nxt, 0)
        i = (i + batch_size*step) % n
        yield X, y

steps_per_epoch = max(1, (len(texto) - SEQ_LEN) // 256)
print('steps_per_epoch (aprox):', steps_per_epoch)


## 4 - Arquitetura do Modelo

LSTM(512) seguida de Dense(vocab, softmax). Entrada one-hot (seq_len × vocab).

In [ ]:
from tensorflow.keras import models, layers, optimizers
model = models.Sequential([
    layers.LSTM(512, input_shape=(SEQ_LEN, vocab)),
    layers.Dense(vocab, activation='softmax'),
])
optimizer = optimizers.Adam(learning_rate=2e-3, clipnorm=1.0)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()


## 5 - Treinamento

Callbacks: ModelCheckpoint, ReduceLROnPlateau e EarlyStopping. Parâmetros sugeridos: BATCH_SIZE=256, EPOCHS=40.

In [ ]:
from tensorflow.keras import callbacks as kb
BATCH_SIZE = 256
EPOCHS = 40
monitor='loss'
cb = [
    kb.ModelCheckpoint(str(MODELO_OUT), save_best_only=True, monitor=monitor, mode='min'),
    kb.ReduceLROnPlateau(monitor=monitor, factor=0.5, patience=3, min_lr=5e-5),
    kb.EarlyStopping(monitor=monitor, patience=5, restore_best_weights=True),
]
gen = batch_generator(texto, char_to_id, seq_len=SEQ_LEN, batch_size=BATCH_SIZE, step=1)
steps_per_epoch = max(1, (len(texto) - SEQ_LEN) // BATCH_SIZE)
history = model.fit(gen, steps_per_epoch=steps_per_epoch, epochs=EPOCHS, callbacks=cb, verbose=1)
print('Treinamento concluido! Melhor modelo salvo em:', MODELO_OUT)


## 6 - Salvamento e Carregamento

Recarrega o modelo/mapeamentos para gerar texto sem retreinar.

In [ ]:
import tensorflow as tf, pickle
modelo = tf.keras.models.load_model(MODELO_OUT) if MODELO_OUT.exists() else model
with open(MAPEAMENTOS_OUT, 'rb') as f:
    maps = pickle.load(f)
c2i = maps['char_to_id']
i2c = maps['id_to_char']
print('Artefatos prontos para gerar texto.')


## 7 - Geracao de Texto

Geração com top-k (k=20) e temperatura 0.8. Use seed (~160 chars) do corpus.

In [ ]:
import numpy as np
def sample_top_k(probs, k=20, temperature=0.8, rng=None):
    probs = np.asarray(probs, dtype=np.float64)
    if rng is None: rng = np.random.default_rng()
    k = int(max(1, min(k, probs.size)))
    top_idx = np.argpartition(-probs, k-1)[:k]
    sel = probs[top_idx]
    if temperature > 0:
        logits = np.log(np.maximum(sel, 1e-9)) / temperature
        logits -= logits.max()
        sel = np.exp(logits)
    p = sel / sel.sum()
    return int(top_idx[rng.choice(len(top_idx), p=p)])

def gerar_texto(model, char_to_id, id_to_char, seed, comprimento=400, k=20, temperatura=0.8):
    vocab = len(id_to_char)
    seq_len = model.input_shape[1] if isinstance(model.input_shape, (list,tuple)) else 160
    rep = ' ' if ' ' in char_to_id else next(iter(char_to_id.keys()))
    seed = ''.join(ch if ch in char_to_id else rep for ch in seed)
    ids = [char_to_id[ch] for ch in seed][-seq_len:]
    if len(ids) < seq_len:
        pad = char_to_id.get(' ', ids[0] if ids else 0)
        ids = [pad] * (seq_len - len(ids)) + ids
    out = []
    rng = np.random.default_rng(42)
    for _ in range(comprimento):
        X = np.zeros((1, seq_len, vocab), dtype=np.float32)
        for t, idx in enumerate(ids):
            if idx < vocab: X[0, t, idx] = 1.0
        p = model.predict(X, verbose=0)[0]
        nxt = sample_top_k(p, k=k, temperature=temperatura, rng=rng)
        out.append(id_to_char.get(nxt, '?'))
        ids = ids[1:] + [nxt]
    return ''.join(out)

# Exemplo: seed = '...'
# print(gerar_texto(modelo, c2i, i2c, seed, comprimento=400, k=20, temperatura=0.8))
